<a href="https://colab.research.google.com/github/imamcs19/SIET2019-DeepN360/blob/master/DeepN360GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 7633660939757796330, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 7222365895580387582
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 13171087451023195899
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14892338381
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 785666688982504263
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"]

In [24]:
# clear all variable by force
#from IPython import get_ipython;   
#get_ipython().magic('reset -sf') 

#from IPython import get_ipython
#def __reset_n_clear__(): 
#    get_ipython().magic('clear')
#    get_ipython().magic('reset -sf')
#    #get_ipython().magic('reset -f')
    

#__reset_n_clear__()

# clear screen
#print ("\n"*80)
#import os
#clear = lambda: os.system('cls')  # On Windows System
#clear()



def Getfitur(string_a,string_b):
    # get fitur yang sama dalam kelompok satu kelas, yang tidak sama tandai dengan "-"
    # misal dari kelas 0, data ke-0 dan ke-1
    """ contoh get index match
    import itertools
    string_a = 'hello_world'
    string_b = 'hi_low_old'
    
    same = [ i for i,x in enumerate(itertools.izip(string_a,string_b)) if all(y==x[0] for y in x)]
    
    In [38]: same
    Out[38]: [0, 3, 4, 7]
    
    """
    #string_a = 'hello_world'
    #string_b = 'hi_low_old'
    
    #string_a = dataFitur[0]
    #string_b = dataFitur[1]
    
    ##Idxsame = [ i for i,x in enumerate(zip(string_a,string_b)) if all(y==x[0] for y in x)]
    Idxnotsame = [ i for i,x in enumerate(zip(string_a,string_b)) if not all(y==x[0] for y in x)]
    
    
    ##print('Indeks Match 2 Str : ',Idxsame, '(',len(Idxsame),')','\n')
    ##print('Indeks notMatch 2 Str : ',Idxnotsame, '(',len(Idxnotsame),')','\n')
    
    ##getcharSame=[string_a[i] for i in Idxsame]
    #print('getcharSame : ',getcharSame,'\n')
    
    ##getcharNotSame_a=[string_a[i] for i in Idxnotsame]
    #print('getcharNotSame_a : ',getcharNotSame_a,'\n')
    
    ##getcharNotSame_b=[string_b[i] for i in Idxnotsame]
    #print('getcharNotSame_b : ',getcharNotSame_b,'\n')
    
    # mereplace value dari list(string_b) dengan '-' dgn index Idxnotsame, hasilnya disimpan dalam ListFrom_string_b
    ListFrom_string_b=[item if index not in Idxnotsame else '-' for index, item in enumerate(list(string_b))]
    getcharSameWithFlag = ''.join(ListFrom_string_b)
    ##print('getcharSameWithFlag : ',getcharSameWithFlag,'\n') # as fitur generate from match
    
    return getcharSameWithFlag



# -*- coding: utf-8 -*-
"""
Created on Thu Jun  9 10:39:02 2016

@author: ImamCS
"""
#import numpy as np
from collections import deque, Counter
#import itertools
from itertools import combinations
from datetime import datetime
import copy
import time
start_time = time.time()
from datetime import timedelta

# baca data fitur dan kelas
dataNkelas = open('dataCoba.txt').readlines()

#get data fitur
#dataN=dataNkelas[0::2] #some_list[start:stop:step]
dataFitur=list(map(str.strip,dataNkelas[0::2])) # some_list[start:stop:step] & remove \n
print('Contoh data ke-0 : ',dataFitur[0],'\n')

jumData=len(dataFitur)
print('Jumlah Data (jumData) : ',jumData,'\n')


#get kelas data
#kelasData=dataNkelas[1::2]
kelasData=list(map(str.strip,dataNkelas[1::2])) #remove \n
#print(kelasData[0])

#get list kelas 
#uniqkelas = []
#[uniqkelas.append(x) for x in Nkelas if x not in uniqkelas]
uniqkelas=list(sorted(set(kelasData)))
print('Unik Kelas : ',uniqkelas,'\n')

jumKelas=len(uniqkelas)
print('Jumlah Kelas (jumKelas) : ', jumKelas, '\n')

#jumlah data tiap kelas
counterkelasNjumFrek=Counter(kelasData)
#print(counterkelasNjumFrek,'\n')

jumTiapKelas=list(counterkelasNjumFrek.values())
#print(jumTiapKelas,'\n')

uniqkelas2=list(sorted(counterkelasNjumFrek.keys())) #saya dengan hasil uniqkelas
#print(uniqkelas2,'\n')

kelasNjumFrek=list(sorted(counterkelasNjumFrek.most_common()))
print('Kelas dan Jumlah Kemunculan : ', kelasNjumFrek,'\n')

# melakukan putaran pada data fitur
# dataFiturIn = ... # data fitur dengan nilai tetap (tdk di-rotate)
# dataFiturFlip = ... # data fitur yang selalu di-rotate sebanyak n-1
jum_rotate = len(dataFitur[0])-1
MinSpesifyRotate = 0
MaxSpesifyRotate = copy.deepcopy(jum_rotate)

catchFitur=[]

GetFiturAwal=41
GetFiturAkhir=101

for SpesifyRotate in range(GetFiturAwal,GetFiturAkhir):

  ## 0. get data grup kelas ke-i
  for class_i in uniqkelas:
      ke_i=int(class_i)
      #print('Kelas ke_i I : ',ke_i,'\n')
      kelasKe=int(uniqkelas[ke_i]) # dari uniqkelas
      print('Kelas ke -',kelasKe,':\n')
      print('============================\n')

      # get data tiap kelas dengan iterasi
      if ke_i==0:
          dataFiturTiapKelas = dataFitur[0:kelasNjumFrek[kelasKe][1]] # utk kelas ke-0
      else:
          dataFiturTiapKelas = dataFitur[kelasNjumFrek[kelasKe-1][1]+1:kelasNjumFrek[kelasKe-1][1]+kelasNjumFrek[kelasKe][1]+1] # utk kelas ke-i


      # cek banyak data tiap kelas
      jumdataFiturTiapKelas=len(dataFiturTiapKelas)
      print('Jumlah data kelas ke - ',ke_i,' : ',jumdataFiturTiapKelas,'\n')


      ## 1. get indeks untuk dataFiturIn dan dataFiturFlip
      startRange=0
      endRange=kelasNjumFrek[kelasKe][1]
      list_idx = [i for i in range (0,endRange)] # indeks untuk dataFitur
      byk_pair = 2

      #different_num = [(a, b) for a in list_idx1 for b in list_idx2 if a != b) # ada yang sama
      diff_pairIdx = [",".join(map(str, comb)) for comb in combinations(list_idx, byk_pair)]
      jum_diff_pairIdx = len(diff_pairIdx)
      print('byak diff_pairIdx : ',len(diff_pairIdx),'\n')

      ## hanya untuk test jum_diff_pairIdx
      #jum_diff_pairIdx=2

      for i in range(0,jum_diff_pairIdx):
          #split by comma dan convert diff_pairIdx to integer
          TiapPairIdx=diff_pairIdx[i].split(',')
          IdxdataFiturIn=int(TiapPairIdx[0])
          IdxdataFiturFlip=int(TiapPairIdx[1])

          #dataFiturIn = ... # data fitur dengan nilai tetap (tdk di-rotate)
          dataFiturIn = dataFiturTiapKelas[IdxdataFiturIn]

          #dataFiturFlip = ... # data fitur yang selalu di-rotate sebanyak n-1
          dataFiturFlip = dataFiturTiapKelas[IdxdataFiturFlip]
          dataFiturFlipGo = deque(dataFiturFlip)

          ## hanya untuk test jum_rotate        
          ##SpesifyRotate=9 #MaxSpesifyRotate # [0;len(dataFitur[0])-1]
          jum_rotate=copy.deepcopy(SpesifyRotate)

          # lakukan rotate untuk dataFiturFlip
          for j in range(0,jum_rotate+1):
              if j==0: # tidak ada rotate
                  #catchFitur[class_i][i][j] = 
                  if j == SpesifyRotate:
                      catchFitur.append(Getfitur(dataFiturIn,dataFiturFlip))
                  #print('catchFitur : ',catchFitur,'\n')
              else:                
                  dataFiturFlipGo.rotate(-1)
                  dataFiturFlipGo_str=''.join(dataFiturFlipGo)
                  #catchFitur[class_i][i][j] = 
                  if j == SpesifyRotate:
                      catchFitur.append(Getfitur(dataFiturIn,dataFiturFlipGo_str))


      ##time.strftime("%a, %d %b %Y %H:%M:%S")
      initwaktu=str(datetime.now())
      getwaktu=initwaktu.replace(" ",'').replace(":","").replace("-","")

      # simpan fitur
      # Create a local file to upload.
      if len(catchFitur)>0:
        with open('/tmp/fitur kelas-'+str(class_i)+'-R'+str(SpesifyRotate)+'-'+getwaktu+'.txt', 'w') as file_handler:
          file_handler.write('\n'.join(str(item) for item in catchFitur)) # if dont want to add "\n" for the last item
          #f.write('my sample file')

      #print('/tmp/to_upload.txt contains:')
      #!cat /tmp/to_upload.txt

      #filepath='./output/fitur kelas-'+str(class_i)+'-'+getwaktu+'.txt'
      #with open(filepath, 'w') as file_handler:
          #for item in the_list:
          #file_handler.write("{}\n".format(item))

      #    file_handler.write('\n'.join(str(item) for item in catchFitur)) # if dont want to add "\n" for the last item

      catchFitur = []

  #print("--- %s seconds ---" % (time.time() - start_time))
  lamawaktueksekusi=str(timedelta(seconds=(time.time() - start_time)))
  print("Lama Waktu Eksekusi : " + lamawaktueksekusi)
  print('\nDone !') 

Contoh data ke-0 :  MEEPQSDPSVEPPLSQETFSDLWKLLPENNVLSPLPSQAMDDLMLSPDDIEQWFTEDPGPDEAPRMPEAAPPVAPAPAAPTPAAPAPAPSWPLSSSVPSQKTYQGSYGFRLGFLHSGTAKSVTCTYSPALNKMFCQLAKTCPVQLWVDSTPPPGTRVRAMAIYKQSQHMTEVVRRCPHHERCSDSDGLAPPQHLIRVEGNLRVEYLDDRNTFRHSVVVPYEPPEVGSDCTTIHYNYMCNSSCMGGMNRRPILTIITLEDSSGNLLGRNSFEVRVCACPGRDRRTEEENLRKKGEPHHELPPGSTKRALPNNTSSSPQPKKKPLDGEYFTLQIRGRERFEMFRELNEALELKDAQAGKEPGGSRAHSSHLKSKKGQSTSRHKKLMFKTEGPDSD 

Jumlah Data (jumData) :  753 

Unik Kelas :  ['0', '1', '2', '3'] 

Jumlah Kelas (jumKelas) :  4 

Kelas dan Jumlah Kemunculan :  [('0', 335), ('1', 128), ('2', 144), ('3', 146)] 

Kelas ke - 0 :


Jumlah data kelas ke -  0  :  335 

byak diff_pairIdx :  55945 

Kelas ke - 1 :


Jumlah data kelas ke -  1  :  128 

byak diff_pairIdx :  8128 

Kelas ke - 2 :


Jumlah data kelas ke -  2  :  144 

byak diff_pairIdx :  10296 

Kelas ke - 3 :


Jumlah data kelas ke -  3  :  146 

byak diff_pairIdx :  10585 

Lama Waktu Eksekusi : 0:01:59.559625

Done !
Kelas ke - 0 :


Jumlah data ke

In [5]:
!pwd

/content


In [25]:
!zip -r ./datafiturR41-100-dari-392.zip /tmp #setelah runu ini dgn shift-enter, lalu klik refresh pada menu colab

  adding: tmp/ (stored 0%)
  adding: tmp/fitur kelas-0-R42-20190616094429.650648.txt (deflated 99%)
  adding: tmp/fitur kelas-3-R65-20190616103406.758045.txt (deflated 99%)
  adding: tmp/fitur kelas-0-R81-20190616110937.659433.txt (deflated 99%)
  adding: tmp/fitur kelas-1-R75-20190616105600.164313.txt (deflated 99%)
  adding: tmp/fitur kelas-1-R43-20190616094640.119512.txt (deflated 99%)
  adding: tmp/fitur kelas-1-R94-20190616114042.924256.txt (deflated 99%)
  adding: tmp/fitur kelas-1-R63-20190616102912.467121.txt (deflated 99%)
  adding: tmp/fitur kelas-0-R74-20190616105330.693432.txt (deflated 99%)
  adding: tmp/fitur kelas-0-R70-20190616104431.546630.txt (deflated 99%)
  adding: tmp/fitur kelas-1-R61-20190616102443.533401.txt (deflated 99%)
  adding: tmp/fitur kelas-0-R69-20190616104214.175178.txt (deflated 99%)
  adding: tmp/fitur kelas-0-R89-20190616112824.766000.txt (deflated 99%)
  adding: tmp/fitur kelas-0-R67-20190616103749.992417.txt (deflated 99%)
  adding: tmp/fitur kela

In [0]:
!rm /tmp/*.txt